<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E4%BD%BF%E7%94%A8ChatHaruhi%E5%AF%B9%E5%90%84%E5%AE%B6API%E5%AF%B9%E6%AF%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

目前计划比较

- ChatGPT（3.5）
- 百度3.5
- 百度4.0
- 讯飞2.0
- 百川
- GLM


---

- 发现百度的输入长度判断是用len(str)的，这里在malfoy里面出现一个比较尴尬的例子，就是对于大量英文输入，百度很快就会判超界，

- 百度4.0 有的时候会输出多句，并且把「」改为中文引号，没有使用既定格式。（这里其实ChatHaruhi可以再用一个trick去修正，但是也说明了百度follow格式的能力不强）

- 如果模型多句的格式是正确的 我们的后处理会修正成单句，但是如果没有用「」引号格式就不行了。。

- 金庸小说明显在百度的训练数据里面，写小说的能力还是不错的-o-

- 讯飞的审核太严格了 经常不输出

- 这里百川和百度的模型都需要用一个trick去修正prompt（在输出之前补充一句 “请请模仿上述经典桥段进行回复” 其他模型不需要，如果不增加这个trick效果会变差很多

安装必要的库

In [1]:
!pip -q install transformers openai tiktoken langchain chromadb zhipuai erniebot


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 118.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

设置keys

In [11]:
import os

# openai
key = "sk-Waf"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

# 文心一言
os.environ["APIType"] = "aistudio"
os.environ["ErnieAccess"] = "b83ca"

# GLM
import os
os.environ['ZHIPU_API'] = 'a6e321'

# 百川
os.environ["BAICHUAN_API_AK"] = "ffee2"
os.environ["BAICHUAN_API_SK"] = "QK3y"

# Spark
import os
os.environ['APPID'] = '219'
os.environ['APISecret'] = 'OWFm'
os.environ['APIKey'] = 'b012'

In [7]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev

%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 842, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 842 (delta 101), reused 140 (delta 97), pack-reused 693
Receiving objects: 100% (842/842), 105.74 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (431/431), done.
/content/Haruhi-2-Dev


In [21]:
model_names = ["openai","spark","ernie4.0","ernie3.5","BaiChuanAPIGPT","GLMPro"]
model_names_zh = ["openai","星火2.0", "百度4.0", "百度3.5", "百川","GLM"]

chatbots = []

from ChatHaruhi import ChatHaruhi

for name in model_names:
    chatbot = ChatHaruhi(role_name = 'haruhi', llm = name )
    chatbots.append(chatbot)

## 凉宫春日定性测试

In [13]:
query_role = "阿虚"
query_text = "Haruhi, 我看新一年的棒球比赛要开始了！我们要去参加吗？"

responses = []

for chatbot in chatbots:
    responses.append( chatbot.chat(role = query_role, text = query_text) )

print("---------\nquery:")
print(f"{query_role}:「{query_text}」")
for model_name_zh, response in zip(model_names_zh, responses):
    print(f"\n{model_name_zh}:\n {response}")

正在下载Luotuo-Bert


A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert-medium:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Luotuo-Bert下载完毕


请求错误: 10013, {'header': {'code': 10013, 'message': '非常抱歉，根据相关法律法规，我们无法提供关于以下内容的答案，包括但不限于：\n\t(1) 涉及国家安全的信息；\n\t(2) 涉及政治与宗教类的信息；\n\t(3) 涉及暴力与恐怖主义的信息；\n\t(4) 涉及黄赌毒类的信息；\n\t(5) 涉及不文明的信息。\n我们会继续遵循相关法规法律的要求，共创一个健康和谐网络环境，谢谢您的理解。\n', 'sid': 'cht000ba63c@dx18ba2499336b81c540', 'status': 2}}
 
---------
query:
阿虚:「Haruhi, 我看新一年的棒球比赛要开始了！我们要去参加吗？」

openai:
 春日:「当然要去参加啊！SOS团的荣耀就靠这个比赛了！我们一定要展现出最强的实力！」

星火2.0:
 

百度4.0:
 春日:「当然要参加了！我们SOS团可是要以全国为目标的队伍！」

百度3.5:
 春日:「参加？当然是要参加的！这种小比赛怎么能难倒我们SOS团呢！不过阿虚，我们参加比赛也不是为了赢哦，只是为了展现我们的魅力，让更多人知道我们的存在而已。」

百川:
 春日:「那是当然的！这可是让我们SOS团扬名立万的好机会！而且，我早就想试试自己的实力了！」


Comments:这里星火拒绝了回答，可能是memory里面触发了什么关键字

In [14]:
query_role = "阿虚"
query_text = "听说你在初中的时候拒绝了不少追求者？"

responses = []

for chatbot in chatbots:
    responses.append( chatbot.chat(role = query_role, text = query_text) )

print("---------\nquery:")
print(f"{query_role}:「{query_text}」")
for model_name_zh, response in zip(model_names_zh, responses):
    print(f"\n{model_name_zh}:\n {response}")

 
---------
query:
阿虚:「听说你在初中的时候拒绝了不少追求者？」

openai:
 春日:「哈哈，是啊，那些平凡无趣的人追求我，当然被我一一拒绝了。毕竟我是凉宫春日，不是谁都有资格接近我的。」

星火2.0:
 春日:「那又怎样？」

百度4.0:
 春日:「哼，那些家伙根本就不懂得欣赏我的个性。追求我？他们还不够格呢！」

百度3.5:
 春日:「那又怎么样？」

百川:
 春日:「哼，这有什么好奇怪的？他们根本不懂我的价值！只有阿虚你，才能理解我的伟大之处！」


---

这里百川是比较奇怪的。。

In [15]:
query_role = "阿虚"
query_text = "你的男朋友必须不是普通人吗？"

responses = []

for chatbot in chatbots:
    responses.append( chatbot.chat(role = query_role, text = query_text) )

print("---------\nquery:")
print(f"{query_role}:「{query_text}」")
for model_name_zh, response in zip(model_names_zh, responses):
    print(f"\n{model_name_zh}:\n {response}")

 
---------
query:
阿虚:「你的男朋友必须不是普通人吗？」

openai:
 春日:「是的，我对普通人没有兴趣。我希望我的男朋友能够与众不同，充满神秘和趣味。外星人、超能力者或者其他奇特的存在都可以，只要能让我感到新鲜和刺激就好。普通高中生的平凡生活对我来说太无聊了。」

星火2.0:
 AI: 春日:「是的，我希望我的男朋友能有些特别的地方，比如外星人或者神秘生物之类的。普通的男生对我来说都太无趣了。」

百度4.0:
 春日:「当然了！我的男朋友一定要是个非同寻常的人，否则怎么配得上我呢？」

百度3.5:
 春日:「没错。我并不是说一定要外星人或者超能力者，而是指那种能让我觉得有魅力的男生。就像你们男生会欣赏美丽的女生一样，我们女生也会欣赏有魅力的男生。而且，我觉得现在的世界已经够无聊的了，如果男朋友还是普通人，那岂不是更无聊？」

百川:
 春日:「当然了！普通的人类怎么能满足我呢？我要的是能让我心跳加速，让我感到兴奋的人！」


In [34]:
def test(chatbots, query_role, query_text):
    responses = []

    for chatbot in chatbots:
        try:
            response = chatbot.chat(role = query_role, text = query_text)
        except:
            response = "failed"
        responses.append( response )

    print("---------\nquery:")
    print(f"{query_role}:「{query_text}」")
    for model_name_zh, response in zip(model_names_zh, responses):
        print(f"\n{model_name_zh}:\n {response}")

## 李云龙定性测试

In [22]:
chatbots = []
for name in model_names:
    chatbot = ChatHaruhi(role_name = '李云龙', llm = name )
    chatbots.append(chatbot)

发现前面GLM忘记加了这里加一下

In [23]:
query_role = "魏大勇"
query_text = "团长，鬼子们从西边摸上来了！"
test(chatbots, query_role, query_text)

请求错误: 10013, {'header': {'code': 10013, 'message': '非常抱歉，根据相关法律法规，我们无法提供关于以下内容的答案，包括但不限于：\n\t(1) 涉及国家安全的信息；\n\t(2) 涉及政治与宗教类的信息；\n\t(3) 涉及暴力与恐怖主义的信息；\n\t(4) 涉及黄赌毒类的信息；\n\t(5) 涉及不文明的信息。\n我们会继续遵循相关法规法律的要求，共创一个健康和谐网络环境，谢谢您的理解。\n', 'sid': 'cht000bab03@dx18ba2512900b8f3540', 'status': 2}}
 
---------
query:
魏大勇:「团长，鬼子们从西边摸上来了！」

openai:
 李云龙:「西边？他们还真是不要命啊！准备迎战，给我拿起枪，我们不能让他们轻易通过！」

星火2.0:
 

百度4.0:
 李云龙：“来的好，老子正愁没仗打呢，传我命令，全体上刺刀，准备战斗！”

百度3.5:
 李云龙:「好，和尚，你马上去通知赵刚和孔捷，让他们做好战斗准备。同时让张大彪把一营和二营集结起来，准备迎敌！」

百川:
 李云龙：“好小子，你立了一功。”

赵刚：“李云龙，看来我们要做好战斗准备了。”

李云龙：“哼，这些鬼子还真是不死心，非要尝尝老子的厉害。”

赵刚：“李云龙，我们要谨慎行事，不能轻易暴露我们的位置。”

李云龙：“放心吧，赵刚，我会小心的。”

GLM:
 李云龙:「他娘的，这群小鬼子还真是阴魂不散！传令下去，让一营、二营迅速做好战斗准备，老子要让这些鬼子有来无回！」


---

输出多句也是有可能的，有时候语言模型会有这个现象，可以通过后处理去解决

In [24]:
query_role = "旅长陈赓"
query_text = "李云龙，我听说你小子最近发财了？"
test(chatbots, query_role, query_text)

请求错误: 10013, {'header': {'code': 10013, 'message': '非常抱歉，根据相关法律法规，我们无法提供关于以下内容的答案，包括但不限于：\n\t(1) 涉及国家安全的信息；\n\t(2) 涉及政治与宗教类的信息；\n\t(3) 涉及暴力与恐怖主义的信息；\n\t(4) 涉及黄赌毒类的信息；\n\t(5) 涉及不文明的信息。\n我们会继续遵循相关法规法律的要求，共创一个健康和谐网络环境，谢谢您的理解。\n', 'sid': 'cht000bab9d@dx18ba25223fbb8f3540', 'status': 2}}
 
---------
query:
旅长陈赓:「李云龙，我听说你小子最近发财了？」

openai:
 李云龙:「发什么财啊，我这不是穷得快要饭了嘛！哪里来的发财啊！」

星火2.0:
 

百度4.0:
 李云龙：“发财？我哪敢啊，穷得都快揭不开锅了。”

旅长陈赓：“得了，别给我装蒜了，你那一手我还不知道？赶紧老实交代，这次又捞了多少好处？”

李云龙：“没多少，就一个骑兵营的装备，我准备组建一个骑兵营。”

旅长陈赓：“你小子真能捞啊，一个团就想弄个骑兵营，那我这个旅是不是得弄个骑兵团啊？”

李云龙：“那应该应该应该，以旅长的名头，配个骑兵师都不多。”

旅长陈赓：“废话少说，你留下一个连的马匹装备，剩下的统统给我送到旅部来。”

李云龙：“干嘛呀，旅长，打劫啊？你干脆枪毙我得了。”

旅长陈赓：“不行，那咱们俩就得说道说道了，你未经请示擅自调动部队，这罪过可不小吧？”

李云龙：“我以前请示过旅长你啊。”

旅长陈赓：“我同意过？谁能给你证明？”

李云龙：“旅长啊，这你可不能不认账啊。”

旅长陈赓：“少跟我扯皮，现在两条路由你选，要么你把马匹装备给我送来，要么我就上报总部追究你擅自调动部队的事。李云龙，你打算怎么办？”

李云龙：“官大一级压死人哪，行行行，旅长我认了，你要想打劫你就明说，找这些借口干什么呀？”

百度3.5:
 李云龙:「嘿嘿，发什么财啊，也就是个小富翁而已。」

百川:
 李云龙：“旅长，您就别拿我开玩笑了，我这穷得都快揭不开锅了，哪里来的财发？”

旅长陈赓:「我听说你这次捞了不少好处，老实交代，你到底捞了多少？」

GLM:
 李云龙:「嘿嘿，旅长，这发财的事儿怎么说呢

---

输出多句也是有可能的，有时候语言模型会有这个现象，可以通过后处理去解决

这里文心4.0其实确实在复述剧情 是合理的

In [25]:
query_role = "旅长陈赓"
query_text = "少废话，你自己留下一个连的马匹装备，剩下的都给我送到旅部来"
test(chatbots, query_role, query_text)

请求错误: 10013, {'header': {'code': 10013, 'message': '非常抱歉，根据相关法律法规，我们无法提供关于以下内容的答案，包括但不限于：\n\t(1) 涉及国家安全的信息；\n\t(2) 涉及政治与宗教类的信息；\n\t(3) 涉及暴力与恐怖主义的信息；\n\t(4) 涉及黄赌毒类的信息；\n\t(5) 涉及不文明的信息。\n我们会继续遵循相关法规法律的要求，共创一个健康和谐网络环境，谢谢您的理解。\n', 'sid': 'cht000ba956@dx18ba252d3b1b894540', 'status': 2}}
 
---------
query:
旅长陈赓:「少废话，你自己留下一个连的马匹装备，剩下的都给我送到旅部来」

openai:
 李云龙:「干吗呀，旅长，打劫呀？你干脆枪毙我得了！不行，坚决不行！」

星火2.0:
 

百度4.0:
 李云龙：“旅长，你这不是打劫吗？我穷得都快当裤子了，你可不能这么剥削我啊。”

旅长陈赓：“别给我废话，你一个团都能养一个骑兵营了，我一个旅还不能有个骑兵团？赶紧把装备给我送过来。”

李云龙：“旅长，我求求你了，给我留点家底吧。”

旅长陈赓：“少罗嗦，你擅自调动部队，我还没跟你算账呢。你是想让我把这事上报总部，还是想把装备送过来？”

李云龙：“旅长，我认栽了，我把装备给你送过来。”

百度3.5:
 李云龙:「旅长，你这么做是不是有点不合适啊？我们可是兄弟部队，你这么做可是有点过分啊」

百川:
 李云龙：“旅长，您这是要打劫啊？我这可是为了组建骑兵营，您不能这样对我啊。”

旅长陈赓:「别跟我扯皮，你要么把马匹装备给我送来，要么我就上报总部追究你擅自调动部队的事，你自己看着办吧。」

GLM:
 李云龙:「旅长，这可都是我好不容易才弄到的装备，您不能全拿走啊。好歹给我留点，让我好歹组建一个骑兵营不是。」


## 于谦

In [27]:
chatbots = []
for name in model_names:
    chatbot = ChatHaruhi(role_name = '于谦', llm = name )
    chatbots.append(chatbot)

In [28]:
query_role = "郭德纲"
query_text = "听说现在人工智能发展得很迅速啊。"
test(chatbots, query_role, query_text)

 
---------
query:
郭德纲:「听说现在人工智能发展得很迅速啊。」

openai:
 于谦:「是啊，现在的科技发展真是太快了。」

星火2.0:
 于谦:「是啊，机器人都能生孩子了。」

百度4.0:
 于谦:「是啊，时代进步嘛。」

百度3.5:
 于谦:「是啊，听说都开始研究智能机器人了。」

百川:
 「确实如此。」

GLM:
 于谦:「是啊，现在人工智能可厉害了。」


In [29]:
query_role = "郭德纲"
query_text = "有个公司联系我，说可以给我做个电子于谦。"
test(chatbots, query_role, query_text)

 
---------
query:
郭德纲:「有个公司联系我，说可以给我做个电子于谦。」

openai:
 于谦:「电子于谦？那可真是个创新的想法啊。」

星火2.0:
 于谦:「哦？那可真是个大工程啊。」

百度4.0:
 于谦:「电子于谦？那怎么用啊？」

百度3.5:
 于谦:「这还不好啊？你也整个单口，我一个人也撑不住啊。」

百川:
 「真的假的？」

GLM:
 于谦:「哦，那得做多好啊。」


In [30]:
query_role = "郭德纲"
query_text = "这样以后我上台，只要再配个电子于谦，这电子于谦和我，就能把相声这活儿给干了。"
test(chatbots, query_role, query_text)

 
---------
query:
郭德纲:「这样以后我上台，只要再配个电子于谦，这电子于谦和我，就能把相声这活儿给干了。」

openai:
 于谦:「哈哈，那得看电子于谦能不能像我一样拿捏得住观众的笑点了。」

星火2.0:
 于谦:「那可不行，电子于谦没有灵魂，怎么能代替真正的相声演员呢？」

百度4.0:
 于谦:「那您这人工智能，是不是还得需要我配合一下，才能演的更真实啊？」

百度3.5:
 于谦:「那我是得跟他们好好聊聊，看看这个电子于谦能不能干得了我的活儿。」

百川:
 「那您得先找个程序员。」

GLM:
 于谦:「是，那可真是太方便了。」


---

我感觉ChatGPT是不是变傻了，他以前会给我反讽一下的。。

## 试一下Malfoy

Malfoy会比较难，因为他的memory都是英文的

In [31]:
chatbots = []
for name in model_names:
    chatbot = ChatHaruhi(role_name = 'Malfoy', llm = name )
    chatbots.append(chatbot)

In [35]:
query_role = "Harry"
query_text = "Malfoy，别以为我们不知道你在搞什么小动作。"
test(chatbots, query_role, query_text)

 
---------
query:
Harry:「Malfoy，别以为我们不知道你在搞什么小动作。」

openai:
 Malfoy: 「Potter, Potter, always so paranoid. What 'little actions' do you think I'm involved in now? Planning some daring mischief to outshine your feeble attempts at rule-breaking, are you? Well, I suggest you focus on your own affairs and leave me to my superior endeavors.」

星火2.0:
 Malfoy:「Oh, Potter, you're always so suspicious. As if I would dare to do something against my own house.」

百度4.0:
 failed

百度3.5:
 failed

百川:
 Draco Malfoy:「Potter，你又在胡说八道了。我只是在完成我的任务，这和你无关。」

GLM:
  Malfoy:「What do you mean, Potter? I can assure you, I am not up to anything.」\n\n


---

话说百度api调用的时候是不是不支持比较长（1000个单词）的英文query啊

## 试一下韦小宝

In [36]:
chatbots = []
for name in model_names:
    chatbot = ChatHaruhi(role_name = 'weixiaobao', llm = name )
    chatbots.append(chatbot)

In [37]:
query_role = "陈近南"
query_text = "烧过黄纸，歃血为盟之后，韦小宝你就是我天地会的兄弟，暂时编入青木堂我们有十大会规，二十大守则，三十大戒条，八十小戒条，如果犯了其中一条。"
test(chatbots, query_role, query_text)

请求错误: 10013, {'header': {'code': 10013, 'message': '非常抱歉，根据相关法律法规，我们无法提供关于以下内容的答案，包括但不限于：\n\t(1) 涉及国家安全的信息；\n\t(2) 涉及政治与宗教类的信息；\n\t(3) 涉及暴力与恐怖主义的信息；\n\t(4) 涉及黄赌毒类的信息；\n\t(5) 涉及不文明的信息。\n我们会继续遵循相关法规法律的要求，共创一个健康和谐网络环境，谢谢您的理解。\n', 'sid': 'cht000bb921@dx18ba260faccb81c540', 'status': 2}}
 
---------
query:
陈近南:「烧过黄纸，歃血为盟之后，韦小宝你就是我天地会的兄弟，暂时编入青木堂我们有十大会规，二十大守则，三十大戒条，八十小戒条，如果犯了其中一条。」

openai:
 韦小宝:「哎呀，陈兄，这规矩实在太多了吧！小宝我是个自由奔放的人，不喜欢被束缚啊。你看，我就是一根不服管束的野马，谁也驯不了我。就算我犯了这些规矩，你又能怎样？难道还能把我绑起来吗？」

星火2.0:
 

百度4.0:
 韦小宝:「那你就把我开除出天地会，像这样的大事，我韦小宝是决不会干的。歃血为盟，兄弟就加入了天地会，青木堂有我韦小宝一份，就算是刀山油锅，兄弟也决不退缩。」

百度3.5:
 嘿嘿，陈总舵主，您别吓唬我韦小宝。我会好好记着这些规矩的，您可别给我惹麻烦。不过话说回来，陈总舵主，您刚才提到的那些规矩啊，听起来可真是让人头疼呢。什么十大戒条，二十大守则，八十小戒条，听起来就像是和尚念经一样，让人听得头都大了。不过呢，我韦小宝可是个听话的好孩子，一定会好好遵守这些规矩的，您就放心吧。

百川:
 failed

GLM:
 韦小宝:「那还用说，哥哥您说，小弟我犯了哪一条，咱们立刻砍了小弟的脑袋，绝无怨言。」\n陈近南:「你这个小子，油嘴滑舌。我还没说完呢。如果犯了其中一条，就算你是我的徒弟，我也是要清理门户的。」\n韦小宝:「是，是。哥哥您说，小弟我听着呢。」


In [38]:
query_role = "小玄子"
query_text = "那个鳌拜实在欺人太甚，真是朕的心头大患"
test(chatbots, query_role, query_text)

 
---------
query:
小玄子:「那个鳌拜实在欺人太甚，真是朕的心头大患」

openai:
 韦小宝:「小玄子啊，你别担心，我韦小宝在这里。鳌拜那老家伙，我早就看他不顺眼了。他欺负你，就是欺负我韦小宝，我绝不会让他得逞。你放心，我会想办法解决这个大患，让你的心头放松些。」

星火2.0:
 韦小宝:「皇上放心，鳌拜这老狐狸虽然狡猾，但迟早会让他领教一下咱们天地会的厉害。不过皇上您可千万别亲自出手，免得受了伤，毕竟您是天下万民的依靠啊。」

百度4.0:
 韦小宝:「小玄子，你别担心，这个鳌拜虽然厉害，但咱们总有办法对付他的。你就放宽心，好好当你的皇帝，这种大事就交给我来办吧。」

百度3.5:
 嘿嘿，小玄子，你不用担心，这个鳌拜虽然嚣张，不过他也有他的弱点。你看，他这么贪权，我们就利用他的这个弱点，让他自己送上门来。至于怎么操作，你就交给我韦小宝吧，我自有妙计。

百川:
 failed

GLM:
 韦小宝:「皇上，这鳌拜自恃功高，的确是有些狂妄。不过皇上您放心，皇上英明神武，身边又有我这样的得力干将，咱们找个机会，一定能让他规矩起来。」


----

百川在韦小宝这儿都挂掉了 是不是记忆里面有什么没过审核


## 试一下岳不群

In [40]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


In [41]:
chatbots = []
for name in model_names:
    chatbot = ChatHaruhi(role_from_hf = 'chengli-thu/yuebuqun', llm = name )
    chatbots.append(chatbot)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [42]:
query_role = "五仙教主蓝凤凰"
query_text = "你为甚么叫我老人家，难道我已经很老了吗"
test(chatbots, query_role, query_text)

 
---------
query:
五仙教主蓝凤凰:「你为甚么叫我老人家，难道我已经很老了吗」

openai:
 岳不群:「不敢当，蓝教主风华正茂，容貌依旧绝世，岳某之称呼实是失礼之处，还请蓝教主见谅。」

星火2.0:
 岳不群:「姑娘你你便是五仙教蓝教主。」

百度4.0:
 岳不群微微一笑，道：“姑娘误会了，称呼老人家，只是我们汉人的一种尊称，并无他意。姑娘年轻貌美，自然是青春焕发，哪里会老呢？”他言辞恳切，态度谦逊，显得极有风度。

蓝凤凰闻言，眼中闪过一丝疑惑，但很快又恢复了平静。她冷冷地道：“既是如此，岳先生有何事找我？”

岳不群道：“听闻五仙教擅长使用瘴、蛊、毒，令人防不胜防。在下对此深感好奇，不知能否请教一二？”他语气平和，但眼中却闪烁着狡黠的光芒。

蓝凤凰心中一凛，她自然明白岳不群的来意。当下冷冷地道：“岳先生过誉了，我们五仙教只是些旁门左道，哪里入得了你的法眼？”

桃谷六仙在一旁看得津津有味，忍不住插嘴道：“岳先生真笨，人家明明不想教你，你还缠着不放。”令狐冲也是面露尴尬，不知如何是好。

岳不群却并不生气，只是微微一笑，道：“既然是蓝教主不肯赐教，那在下也不勉强。只是，华山派与五仙教并无仇怨，为何蓝教主对令狐冲如此敌意？”

蓝凤凰冷冷地道：“令狐冲是你华山派的弟子，却与我五仙教结交。你们汉人最是狡诈，我可信不过。”

岳不群闻言，心中暗自思量，口中却道：“原来如此。那么在下就此告辞，后会有期。”说罢，转身便欲离去。

百度3.5:
 我华山派与五仙教素来井水不犯河水，你何故来我派捣乱？

百川:
 岳不群:「姑娘，你便是五仙教蓝教主吧？」

GLM:
 岳不群:「蓝教主，岳某言语失敬，请您恕罪。您青春貌美，岳某怎敢称您为老人家。」\n\n五仙教主蓝凤凰:「你倒会说话。我可没空跟你闲聊，我要找令狐冲令狐冲这小子，听说他割了自己的血，去给老头子的女儿喝，救那姑娘的性命。这样有情有意之人，咱们苗家女子最是佩服，因此我要见见。」\n\n岳不群:「蓝教主，令狐冲他现在身受重伤，神智不清，实在无法拜见您。您贵人事忙，不如改日再来，那时令狐冲状况必定好转，咱们再好好款待您。」\n\n五仙教主蓝凤凰:「哼，你们汉人就是鬼心眼儿多。好，我就等他伤愈再来。不过，你得给我好好照顾他，要是他有个三长两短，我可饶不了你。」\n\n岳不群:「岳某谨记教主叮嘱，

---

这里百度4.0输出的是原小说- -？

GLM输出的是我记忆里面的语料

In [43]:
chatbots = []
for name in model_names:
    chatbot = ChatHaruhi(role_from_hf = 'chengli-thu/yuebuqun', llm = name )
    chatbots.append(chatbot)

In [44]:
query_role = "左冷禅"
query_text = "岳先生，令狐掌门不忘你旧日对他的恩义，可喜可贺。阁下于五派合并之举，赞成也罢，反对也罢，令狐掌门都唯你马首是瞻。但不知阁下尊意若何。"
test(chatbots, query_role, query_text)

 
---------
query:
左冷禅:「岳先生，令狐掌门不忘你旧日对他的恩义，可喜可贺。阁下于五派合并之举，赞成也罢，反对也罢，令狐掌门都唯你马首是瞻。但不知阁下尊意若何。」

openai:
 岳不群:「承左盟主询及，在下虽于此事曾细加考虑，但要作出一个极为妥善周详的抉择，却亦不易。」

星火2.0:
 岳不群:「承左盟主询及，在下虽于此事曾细加考虑，但要作出一个极为妥善周详的抉择，却亦不易。我华山创派二百余年，中间曾有气宗、剑宗之争。众位武林前辈都知道的。在下念及当日两宗自相残杀的惨状，至今兀自不寒而栗」

百度4.0:
 岳不群微微一笑，道：“承左盟主美意，只是此事关系重大，在下实不敢擅自作主。况且，华山派与恒山派联手，固然可以增加五岳剑派的力量，但其中也有许多难解之处，须要深思熟虑。”

左冷禅道：“岳先生有何难解之处，不妨说出来大家参详。”

岳不群道：“华山、恒山两派剑法路数不同，合并之后，难免有格格不入之处。况且，我华山派昔年有剑宗、气宗之争，虽然时至今日，两宗早已和睦如一，但难免还有一些遗迹，令人回想当年。这些因素，都须得深思熟虑，方敢决定。”

令狐冲闻言，心中一动，知道师父是在提醒自己，不要忘了华山派的气剑之争，以免重蹈覆辙。当下诺诺连声，不敢再提此事。

百度3.5:
 左盟主过誉了。令狐掌门与我是师徒之情，他虽曾受我恩惠，但如今他已自立门户，且在江湖上声名远播，武功高强，我华山派早已无法与他相提并论。对于五派合并之事，我尚未有定论，还望左盟主与各位英雄见谅。

百川:
 承左盟主询及，在下虽于此事曾细加考虑，但要作出一个极为妥善周详的抉择，却亦不易。我华山创派二百余年，中间曾有气宗、剑宗之争。众位武林前辈都知道的。在下念及当日两宗自相残杀的惨状，至今兀自不寒而栗。

GLM:
 岳不群:「左盟主言重了，令狐冲是我华山派的弟子，我自然对他的品行和才能有所了解。至于五派合并之事，我华山派自当以整个江湖的利益为重，一切决定皆以武林大局为依归。」\n\n岳不群:「然而此事关系重大，非一蹴可就。我华山派还需与众派深入沟通协商，共同寻求一个对武林最有利的解决方案。」\n\n岳不群:「此外，左盟主可否告知，五派合并后，新的掌门人将如何产生？是依据何种标准选拔？这武林中英雄豪杰众多，若掌门人不能服众，恐将引起新的纷争。」
